In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
print(datetime.datetime.now())
print(os.getcwd())

ty_start_date=datetime.date(2019,11,3)
ty_end_date=datetime.date(2020,2,1)

ly_start_date=ty_start_date-datetime.timedelta(days=52*7)
ly_end_date=ty_end_date-datetime.timedelta(days=52*7)


print("ty_start_date:",ty_start_date)
print("ty_end_date:",ty_end_date)

print("ly_start_date:",ly_start_date)
print("ly_end_date:",ly_end_date)


2020-02-06 12:45:57.512060
/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_non_rewards_sales_so_far_2019Q4_4_weeks
ty_start_date: 2019-11-03
ty_end_date: 2020-02-01
ly_start_date: 2018-11-04
ly_end_date: 2019-02-02


In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
list_files_ly=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_files_ly=[x for x in list_files_ly if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(ly_start_date).replace("-","")]
list_files_ly=[x for x in list_files_ly if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(ly_end_date).replace("-","")]
list_files_ly.sort()
print(len(list_files_ly))

13


In [8]:
df['item_transaction_amt'].sum()

24.08

In [13]:
# qc last year sales
total_sales=0
rewards_sales=0
non_rewards_sales=0

for f in list_files_ly:
    df=pd.read_table(f,usecols=['customer_id_hashed','item_transaction_amt'],sep="|")
    r=df[pd.notnull(df['customer_id_hashed'])]
    n=df[pd.isnull(df['customer_id_hashed'])]
    
    total_sales+=df['item_transaction_amt'].sum()
    rewards_sales+=r['item_transaction_amt'].sum()
    non_rewards_sales+=n['item_transaction_amt'].sum()
    

In [14]:
rewards_sales

938957721.5799993

In [15]:
non_rewards_sales

657901037.8100001

In [16]:
total_sales

1596858759.3900006

In [3]:
list_files_ty=recursive_file_gen("/home/jian/BigLots/")
list_files_ty=[x for x in list_files_ty if "dailysales" in x.lower()]
list_files_ty=[x for x in list_files_ty if "/MediaStorm_" in x]
list_files_ty=[x for x in list_files_ty if x.split("/MediaStorm_")[1][:10]>=str(ty_start_date)]
list_files_ty=[x for x in list_files_ty if x.split("/MediaStorm_")[1][:10]<=str(ty_end_date)]
list_files_ty.sort()
print(len(list_files_ty))

13


In [4]:
list_files_ty

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-28/MediaStormDailySales20191231-112945-515.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-04/MediaStormDailySales20200107-112859-015.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-11/MediaStormDailySales20200114-115009-140.txt',


In [5]:
df_product_taxo=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                              dtype=str,sep="|")
df_product_taxo=df_product_taxo[['division_id','department_id','class_code_id','subclass_id']].drop_duplicates()
df_product_taxo['division_id']=df_product_taxo['division_id'].astype(int)
df_product_taxo['department_id']=df_product_taxo['department_id'].astype(int)
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
df_department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|")

df_product_taxo.head(2)

,division_id,department_id,class_code_id,subclass_id
0,1,108,11001,2
1,1,108,11001,4


In [6]:
department_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",
                           dtype=str,sep="|")
department_name.head(2)

,department_id,department_desc
0,108,CANDYSNACK
1,109,CANSNPASTA


In [7]:
department_name[department_name['department_desc'].apply(lambda x: "mattress" in x.lower())]
# 615 only 

,department_id,department_desc
64,615,MATTRESSES


In [8]:
df_product_taxo[df_product_taxo['department_id']==615]

,division_id,department_id,class_code_id,subclass_id
2283,9,615,61501,1
2284,9,615,61501,3
2285,9,615,61501,5
2286,9,615,61502,1
2287,9,615,61502,3
2288,9,615,61502,5
2289,9,615,61503,1
2290,9,615,61503,3
2291,9,615,61503,5
2292,9,615,61504,1


In [9]:
pd.options.display.max_rows=99
department_name

,department_id,department_desc
0,108,CANDYSNACK
1,109,CANSNPASTA
2,110,BEVGROCERY
3,111,DIRECT STORE DELIVERIES
4,112,ALCOHOL
5,114,SPECIALTY FOODS
6,115,FROZEN FOOD
7,120,HBC
8,121,CONTROLLED SUBSTANCE
9,130,HOUSEKEEPING


In [10]:
df_summary_InWeek_ly=pd.DataFrame()
df_summary_CumQ_ly=pd.DataFrame()

df_rewards_ids_ly=pd.DataFrame()
df_division_output_ly=pd.DataFrame()
df_department_output_ly=pd.DataFrame()

cum_R_sales=0
cum_R_trans=0

cum_N_sales=0
cum_N_trans=0

week_num=0
for file in list_files_ly:
    week_num+=1
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,df_product_taxo,on=['class_code_id','subclass_id'],how="left")
    df['division_id']=df['division_id'].fillna("nan")
    df['department_id']=df['department_id'].fillna("nan")
    
    date_min=df['transaction_dt'].min()
    date_max=df['transaction_dt'].max()
    
    df['rewards_label']=np.where(pd.notnull(df['customer_id_hashed']),"Rewards","Non_Rewards")
    
    
    ######
    # division view from department, no change of the order below
    df_department_sales=df.groupby(['rewards_label','division_id','department_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_division_sales=df_department_sales.groupby(['rewards_label','division_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_division_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','rewards_label']].drop_duplicates()
    df_division_trans['trans']=1
    df_division_trans=df_division_trans.groupby(['rewards_label','division_id'])['trans'].sum().to_frame().reset_index()
    
    df_department_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','department_id','rewards_label']].drop_duplicates()
    df_department_trans['trans']=1
    df_department_trans=df_department_trans.groupby(['rewards_label','division_id','department_id'])['trans'].sum().to_frame().reset_index()

    df_division=pd.merge(df_division_sales,df_division_trans,on=['rewards_label','division_id'],how="outer")
    df_department=pd.merge(df_department_sales,df_department_trans,on=['rewards_label','division_id','department_id'],how="outer")
    
    df_division['week_num']=week_num
    df_division['week_start']=date_min
    df_division['week_end']=date_max

    df_department['week_num']=week_num
    df_department['week_start']=date_min
    df_department['week_end']=date_max    
    ######
    df_division_output_ly=df_division_output_ly.append(df_division)
    df_department_output_ly=df_department_output_ly.append(df_department)
    
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    num_R_sales=df_rewards['item_transaction_amt'].sum()
    num_R_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates().shape[0]
    num_R_shoppers=df_rewards['customer_id_hashed'].nunique()
    
    num_N_sales=df_nonrewards['item_transaction_amt'].sum()
    num_N_trans=df_nonrewards[['location_id','transaction_dt','transaction_id']].drop_duplicates().shape[0]
    
    df_rewards_shoppers=df_rewards[['customer_id_hashed']].drop_duplicates()
    df_rewards_shoppers['week_start']=date_min
    df_rewards_shoppers['week_end']=date_max
    df_rewards_ids_ly=df_rewards_ids_ly.append(df_rewards_shoppers)
    
    cum_R_sales+=num_R_sales
    cum_R_trans+=num_R_trans
    cum_N_sales+=num_N_sales
    cum_N_trans+=num_N_trans
    cum_R_shoppers=df_rewards_ids_ly['customer_id_hashed'].nunique()
    
    df_in_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":num_R_sales,"Rewards_trans":num_R_trans,"Rewards_shoppers":num_R_shoppers,
                             'NonRewards_sales':num_N_sales,"NonRewards_trans":num_N_trans,
                            },index=[0])
    
    df_cum_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":cum_R_sales,"Rewards_trans":cum_R_trans,"Rewards_shoppers":cum_R_shoppers,
                             'NonRewards_sales':cum_N_sales,"NonRewards_trans":cum_N_trans,
                            },index=[0])

    df_summary_InWeek_ly=df_summary_InWeek_ly.append(df_in_week)
    df_summary_CumQ_ly=df_summary_CumQ_ly.append(df_cum_week)
    print(datetime.datetime.now(),week_num,"done")
        

2020-02-05 15:53:14.884795 1 done
2020-02-05 15:55:29.619537 2 done
2020-02-05 15:58:29.821592 3 done
2020-02-05 16:01:35.555949 4 done
2020-02-05 16:04:59.478780 5 done
2020-02-05 16:08:28.437097 6 done
2020-02-05 16:12:31.110274 7 done
2020-02-05 16:15:18.061515 8 done
2020-02-05 16:17:16.189866 9 done
2020-02-05 16:19:12.172196 10 done
2020-02-05 16:21:27.036539 11 done
2020-02-05 16:23:39.453213 12 done
2020-02-05 16:25:33.053808 13 done


In [11]:
df_summary_InWeek_ty=pd.DataFrame()
df_summary_CumQ_ty=pd.DataFrame()

df_rewards_ids_ty=pd.DataFrame()
df_division_output_ty=pd.DataFrame()
df_department_output_ty=pd.DataFrame()

cum_R_sales=0
cum_R_trans=0

cum_N_sales=0
cum_N_trans=0

week_num=0
for file in list_files_ty:
    week_num+=1
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,df_product_taxo,on=['class_code_id','subclass_id'],how="left")
    df['division_id']=df['division_id'].fillna("nan")
    df['department_id']=df['department_id'].fillna("nan")
    
    date_min=df['transaction_dt'].min()
    date_max=df['transaction_dt'].max()
    
    df['rewards_label']=np.where(pd.notnull(df['customer_id_hashed']),"Rewards","Non_Rewards")
    
    
    ######
    # division view from department, no change of the order below
    df_department_sales=df.groupby(['rewards_label','division_id','department_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_division_sales=df_department_sales.groupby(['rewards_label','division_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_division_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','rewards_label']].drop_duplicates()
    df_division_trans['trans']=1
    df_division_trans=df_division_trans.groupby(['rewards_label','division_id'])['trans'].sum().to_frame().reset_index()
    
    df_department_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','department_id','rewards_label']].drop_duplicates()
    df_department_trans['trans']=1
    df_department_trans=df_department_trans.groupby(['rewards_label','division_id','department_id'])['trans'].sum().to_frame().reset_index()

    df_division=pd.merge(df_division_sales,df_division_trans,on=['rewards_label','division_id'],how="outer")
    df_department=pd.merge(df_department_sales,df_department_trans,on=['rewards_label','division_id','department_id'],how="outer")
    
    df_division['week_num']=week_num
    df_division['week_start']=date_min
    df_division['week_end']=date_max

    df_department['week_num']=week_num
    df_department['week_start']=date_min
    df_department['week_end']=date_max    
    ######
    df_division_output_ty=df_division_output_ty.append(df_division)
    df_department_output_ty=df_department_output_ty.append(df_department)
    
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    num_R_sales=df_rewards['item_transaction_amt'].sum()
    num_R_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates().shape[0]
    num_R_shoppers=df_rewards['customer_id_hashed'].nunique()
    
    num_N_sales=df_nonrewards['item_transaction_amt'].sum()
    num_N_trans=df_nonrewards[['location_id','transaction_dt','transaction_id']].drop_duplicates().shape[0]
    
    df_rewards_shoppers=df_rewards[['customer_id_hashed']].drop_duplicates()
    df_rewards_shoppers['week_start']=date_min
    df_rewards_shoppers['week_end']=date_max
    df_rewards_ids_ty=df_rewards_ids_ty.append(df_rewards_shoppers)
    
    cum_R_sales+=num_R_sales
    cum_R_trans+=num_R_trans
    cum_N_sales+=num_N_sales
    cum_N_trans+=num_N_trans
    cum_R_shoppers=df_rewards_ids_ty['customer_id_hashed'].nunique()
    
    df_in_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":num_R_sales,"Rewards_trans":num_R_trans,"Rewards_shoppers":num_R_shoppers,
                             'NonRewards_sales':num_N_sales,"NonRewards_trans":num_N_trans,
                            },index=[0])
    
    df_cum_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":cum_R_sales,"Rewards_trans":cum_R_trans,"Rewards_shoppers":cum_R_shoppers,
                             'NonRewards_sales':cum_N_sales,"NonRewards_trans":cum_N_trans,
                            },index=[0])

    
    df_summary_InWeek_ty=df_summary_InWeek_ty.append(df_in_week)
    df_summary_CumQ_ty=df_summary_CumQ_ty.append(df_cum_week)
    print(datetime.datetime.now(),week_num,"done")
        

2020-02-05 16:27:22.837792 1 done
2020-02-05 16:29:20.131669 2 done
2020-02-05 16:31:40.044934 3 done
2020-02-05 16:34:53.977408 4 done
2020-02-05 16:38:20.354568 5 done
2020-02-05 16:42:01.478091 6 done
2020-02-05 16:46:08.854462 7 done
2020-02-05 16:49:28.428052 8 done
2020-02-05 16:51:53.890809 9 done
2020-02-05 16:53:57.119716 10 done
2020-02-05 16:56:25.286087 11 done
2020-02-05 16:58:46.999925 12 done
2020-02-05 17:00:43.844175 13 done


In [12]:
df_summary_InWeek_ty=df_summary_InWeek_ty[df_summary_InWeek_ly.columns.tolist()]

writer=pd.ExcelWriter("./BL_sales_summary_in_quarter_11weeks_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_summary_InWeek_ty.to_excel(writer,"df_summary_InWeek_ty",index=False)
df_summary_CumQ_ty.to_excel(writer,"df_summary_CumQ_ty",index=False)
df_division_output_ty.to_excel(writer,"division_output_ty",index=False)
df_department_output_ty.to_excel(writer,"department_output_ty",index=False)

df_summary_InWeek_ly.to_excel(writer,"df_summary_InWeek_ly",index=False)
df_summary_CumQ_ly.to_excel(writer,"df_summary_CumQ_ly",index=False)
df_division_output_ly.to_excel(writer,"division_output_ly",index=False)
df_department_output_ly.to_excel(writer,"department_output_ly",index=False)

df_division_name.to_excel(writer,"df_division_name",index=False)
df_department_name.to_excel(writer,"df_department_name",index=False)
writer.save()

In [13]:
df_rewards_ids_ty.to_csv("./df_rewards_ids_ty_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
df_rewards_ids_ly.to_csv("./df_rewards_ids_ly_JL_"+str(datetime.datetime.now().date())+".csv",index=False)